# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [43]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium
import numpy as np

# Import API key
from folium.plugins import HeatMap
from api_keys import geoapify_key

In [45]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,thompson,55.7435,-97.8558,2.09,100,40,0.00,CA,1728354323
1,1,praia,14.9215,-23.5087,25.30,88,20,3.09,CV,1728354346
2,2,ushuaia,-54.8000,-68.3000,5.81,87,0,2.57,AR,1728354347
3,3,port-aux-francais,-49.3500,70.2167,2.77,71,25,10.45,TF,1728354317
4,4,blackmans bay,-43.0167,147.3167,10.41,71,99,0.89,AU,1728354349


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [48]:
#%%capture --no-display
# Configure the map plot
# YOUR CODE HERE


m = folium.Map(location=[30.0, 31.0], zoom_start=2)

locations = city_data_df[['Lat', 'Lng']].values.tolist()
cities= city_data_df['City'].values.tolist()
humidity = city_data_df['Humidity'].values.tolist()

for (Lat, Lng), city, hum in zip(locations, cities, humidity):
    folium.CircleMarker(
        location = (Lat, Lng),
        radius=hum/10,  # Adjust the size based on humidity, modify the divisor as needed
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=f"{city}: {hum}% Humidity"
    ).add_to(m)

m

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [51]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather = (city_data_df['Max Temp'] > 22) & (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)
ideal_weather_df = city_data_df[ideal_weather]

# Drop any rows with null values
cleaned_ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
cleaned_ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,52,marat,25.0701,45.4615,22.88,19,0,2.83,SA,1728354403
96,96,san patricio,28.0170,-97.5169,25.97,56,0,3.88,US,1728354453
151,151,fort bragg,35.1390,-79.0060,22.35,63,0,2.06,US,1728354515
152,152,williamson,30.6502,-97.6003,25.75,52,0,2.06,US,1728354516
166,166,san angelo,31.4638,-100.4370,23.30,50,0,2.06,US,1728354533
331,331,remire-montjoly,4.9167,-52.2667,26.02,94,0,1.03,GF,1728354722
431,431,nushki,29.5542,66.0215,22.94,34,0,2.28,PK,1728354834
452,452,galveston,29.3669,-94.9669,25.70,70,0,0.00,US,1728354857
498,498,tobruk,32.0836,23.9764,22.18,70,0,3.77,LY,1728354909
542,542,jiwani,25.0500,61.7417,26.94,73,0,0.95,PK,1728354960


### Step 3: Create a new DataFrame called `hotel_df`.

In [54]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame({
           'City': city_data_df['City'], 
           'Country' : city_data_df['Country'], 
           'Lat' : city_data_df['Lat'], 
           'Lng' : city_data_df['Lng'],
           'Humidity' : city_data_df['Humidity']})

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
0,thompson,CA,55.7435,-97.8558,100,
1,praia,CV,14.9215,-23.5087,88,
2,ushuaia,AR,-54.8000,-68.3000,87,
3,port-aux-francais,TF,-49.3500,70.2167,71,
4,blackmans bay,AU,-43.0167,147.3167,71,
...,...,...,...,...,...,...
567,crane,US,31.3974,-102.3501,33,
568,salair,RU,54.2312,85.7972,69,
569,strezhevoy,RU,60.7333,77.5889,73,
570,kruisfontein,ZA,-34.0033,24.7314,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [57]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories': "accomodation.hotel",  
          'apiKey': geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

   
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{['Longitude'], ['Latitude']},{radius}"
    params["bias"] = f"proximity:{['Longitude'], ['Latitude']}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
thompson - nearest hotel: No hotel found
praia - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
karasburg - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
st. john's - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
turaif - nearest hotel: No hotel found
susuman - nearest hotel: No hotel found
trabzon - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
gebeit - nearest hotel: No hotel found
praia da vitoria - nearest hotel: No hotel found
isafjordur - nearest hotel: No hotel found
cabedelo - nearest hotel: No hotel found
mana - nearest hotel: No hotel found
palangkaraya - nearest hotel: No hotel found
eenhana - n

,City,Country,Lat,Lng,Humidity,Hotel Name
0,thompson,CA,55.7435,-97.8558,100,No hotel found
1,praia,CV,14.9215,-23.5087,88,No hotel found
2,ushuaia,AR,-54.8000,-68.3000,87,No hotel found
3,port-aux-francais,TF,-49.3500,70.2167,71,No hotel found
4,blackmans bay,AU,-43.0167,147.3167,71,No hotel found
...,...,...,...,...,...,...
567,crane,US,31.3974,-102.3501,33,No hotel found
568,salair,RU,54.2312,85.7972,69,No hotel found
569,strezhevoy,RU,60.7333,77.5889,73,No hotel found
570,kruisfontein,ZA,-34.0033,24.7314,79,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [60]:
%%capture --no-display
# Configure the map plot
m = folium.Map(location=[30.0, 31.0], zoom_start=2)

locations = hotel_df[['Lat', 'Lng']].values.tolist()
hotel_names= hotel_df['Hotel Name'].values.tolist()
countries = hotel_df['Country'].values.tolist()

for loc, hotel, country in zip(locations, hotel_names, countries):
    folium.Marker(
        location=loc,
        popup=f"Hotel: {hotel}<br>Country: {country}",
        icon=folium.Icon(color='blue') 
    ).add_to(m)
m